In [1]:
#Import packages
import pandas as pd#Return the Uniprot id for querying json entries in Uniprot, or "" if not found
def get_uniprot_id(ensembl_id):
    #Querying the Biomart database to get number of matches
    num = ens_genes.count({
        'filters': {
                'ensembl_gene_id': ensembl_id
        },
        'attributes': [
                'uniprotswissprot'
        ]
    })

    #In case no match is found
    if (num == 0):
        return ""

    #In case of at least one match: querying the Biomart database
    response = ens_genes.search({
        'filters': {
                'ensembl_gene_id': ensembl_id
        },
        'attributes': [
                'uniprotswissprot'
        ]
    })

    #Reading the query result
    uniprot_id = ""
    for line in response.iter_lines():
        line = line.decode('utf-8')
        uniprot_id = (line.split("\t")[0]).encode('ascii','ignore')
        break
    
    return uniprot_id
import numpy as np
from biomart import BiomartServer
import json
import pickle
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>");

In [2]:
#Return the Uniprot id for querying json entries in Uniprot, or "" if not found
def get_uniprot_id(ensembl_id):
    #Querying the Biomart database to get number of matches
    num = ens_genes.count({
        'filters': {
                'ensembl_gene_id': ensembl_id
        },
        'attributes': [
                'uniprotswissprot'
        ]
    })

    #In case no match is found
    if (num == 0):
        return ""

    #In case of at least one match: querying the Biomart database
    response = ens_genes.search({
        'filters': {
                'ensembl_gene_id': ensembl_id
        },
        'attributes': [
                'uniprotswissprot'
        ]
    })

    #Reading the query result
    uniprot_id = ""
    for line in response.iter_lines():
        line = line.decode('utf-8')
        uniprot_id = (line.split("\t")[0]).encode('ascii','ignore')
        break
    
    return uniprot_id

In [3]:
def find_longest_prot(prot_ids):
    max_len = 0
    max_prot_id = ""
    for prot in prot_ids:
        prot_lens = (gene_table[gene_table["prot"] == prot]["length"]).unique()
        if (len(prot_lens) > 1):
            print "Error: more than one length fir protein id: "+prot #Sanity check
        if (prot_lens[0] > max_len):
            max_len = prot_lens[0]
            max_prot_id = prot
    return max_prot_id

In [4]:
#Reading the domain data
curr_dir = !pwd
pfam_version = "32"
in_path = curr_dir[0]+"/../3.parse_HMMER/hmm_domains/pfam-v"+pfam_version+"/"
out_path = curr_dir[0]+"/domains_canonic_prot/pfam-v"+pfam_version+"/"
domains_files = !ls $in_path

#Importing Biomart database
server = BiomartServer("http://grch37.ensembl.org/biomart")
ens_genes = server.datasets[u'hsapiens_gene_ensembl']

In [5]:
len(domains_files[:])

6510

In [21]:
domains_files.index("zf-C3HC4_5.csv")

6392

In [11]:
domains_files[951]

'CLPTM1.csv'

In [22]:
%%time
print "Starting..."
for dom_filename in domains_files[6393:]:
    
    domain_data = pd.read_csv(in_path+dom_filename, sep='\t', index_col=0)
    canonic_protein = {}
    genes_ids = (domain_data["gene"]).unique()
    no_uniprot = 0
    no_canonic_len = 0
    
    for i in range(0,len(genes_ids)):
        ens_gene = genes_ids[i]
        #print "ens_gene = "+ens_gene
        gene_table = domain_data[domain_data["gene"] == ens_gene]
        protein_ids = (gene_table["prot"]).unique()

        if (len(protein_ids) == 1):
            #Saving the one protein id available
            canonic_protein[ens_gene] = protein_ids[0]

        #If there's more then one transcript: finding what's the canonic protein length from uniprot
        else:
            ens_gene_major_number = ens_gene[:ens_gene.find(".")]
            uniprot_id = get_uniprot_id(ens_gene_major_number)
            if (uniprot_id == ""):
                #print ens_gene+": uniprot wasn't found for "+ens_gene
                canonic_protein[ens_gene] = find_longest_prot(protein_ids)
                no_uniprot += 1
                continue
            uniprot_url = "http://togows.dbcls.jp/entry/uniprot/"+uniprot_id+".json"
            try:
                uniprot_json = pd.read_json(uniprot_url)
                canonic_len = uniprot_json.aalen[0]
            except:
                canonic_protein[ens_gene] = find_longest_prot(protein_ids)
                no_uniprot += 1
                continue

            found = False
            for prot in protein_ids:
                prot_lens = (gene_table[gene_table["prot"] == prot]["length"]).unique()
                if (len(prot_lens) > 1):
                    print "Error: more than one length for protein id: "+prot #Sanity check

                #If the length equal the canonical, this is the canonical protein id
                if (prot_lens[0] == canonic_len):
                    found = True
                    canonic_protein[ens_gene] = prot
                    break

            if (found == False):
                #print ens_gene+": Proteins don't match the Uniprot canonic"
                no_canonic_len += 1
                canonic_protein[ens_gene] = find_longest_prot(protein_ids)
                
    domain_sym = dom_filename[:dom_filename.find(".")]
    with open(out_path+domain_sym+"_canonic_prot.pik", 'wb') as handle:
        pickle.dump(canonic_protein, handle, protocol=pickle.HIGHEST_PROTOCOL)  
        
    print "Finished "+dom_filename
    

Starting...
Finished zf-C3HC4.csv
Finished zf-C3HC.csv
Finished zf-C4.csv
Finished zf-C4H2.csv
Finished zf-C4pol.csv
Finished zf-C4_Topoisom.csv
Finished zf-C5HC2.csv
Finished zf-C6H2.csv
Finished zf-CCCH_2.csv
Finished zf-CCCH_3.csv
Finished zf_CCCH_4.csv
Finished zf-CCCH_4.csv
Finished zf_CCCH_5.csv
Finished zf-CCCH_8.csv
Finished zf-CCCH.csv
Finished zf-CCHC_2.csv
Finished zf-CCHC_3.csv
Finished zf-CCHC_6.csv
Finished zf-CCHC.csv
Finished zf-CCHH.csv
Finished zf-CDGSH.csv
Finished zf-CHCC.csv
Finished zf-CHY.csv
Finished zf-CpG_bind_C.csv
Finished zf-CRD.csv
Finished zf-CSL.csv
Finished zf-CW.csv
Finished zf-CXXC.csv
Finished zf-DBF.csv
Finished zf-Di19.csv
Finished zf-DNA_Pol.csv
Finished zf-DNL.csv
Finished zf-dskA_traR.csv
Finished zf-FCS.csv
Finished zf-FPG_IleRS.csv
Finished zf-GRF.csv
Finished zf-H2C2_2.csv
Finished zf-H2C2_5.csv
Finished zf_Hakai.csv
Finished zf-HC5HC2H_2.csv
Finished zf-HC5HC2H.csv
Finished zf-HIT.csv
Finished zf-LITAF-like.csv
Finished zf-LYAR.csv
Finished 

In [6]:
canonic_protein

{'ENSG00000100601.5': 'ENSP00000216489.3',
 'ENSG00000125652.3': 'ENSP00000245812.2',
 'ENSG00000137760.10': 'ENSP00000374219.3',
 'ENSG00000166199.8': 'ENSP00000302232.4',
 'ENSG00000189046.6': 'ENSP00000343021.3',
 'ENSG00000239382.6': 'ENSP00000252984.6'}

In [12]:
domain_data.shape

(1317, 25)

### draft - old code - don't run

In [36]:
#Create a dictionary of the canonical transcipt ids and protein ids
#canonic_transcript = {}
canonic_protein = {}
genes_ids = (domain_data["gene"]).unique()
no_uniprot = 0
no_canonic_len = 0

In [37]:
for i in range(0,len(genes_ids)):
    ens_gene = genes_ids[i]
    gene_table = domain_data[domain_data["gene"] == ens_gene]
    protein_ids = (gene_table["prot"]).unique()

    if (len(protein_ids) == 1):
        #Saving the one protein id available
        canonic_protein[ens_gene] = protein_ids[0]

    #If there's more then one transcript: finding what's the canonic protein length from uniprot
    else:
        ens_gene_major_number = ens_gene[:ens_gene.find(".")]
        uniprot_id = get_uniprot_id(ens_gene_major_number)
        if (uniprot_id == ""):
            print ens_gene+": uniprot wasn't found for "+ens_gene
            canonic_protein[ens_gene] = find_longest_prot(protein_ids)
            no_uniprot += 1
            continue
        uniprot_url = "http://togows.dbcls.jp/entry/uniprot/"+uniprot_id+".json"
        uniprot_json = pd.read_json(uniprot_url)
        canonic_len = uniprot_json.aalen[0]

        found = False
        for prot in protein_ids:
            prot_lens = (gene_table[gene_table["prot"] == prot]["length"]).unique()
            if (len(prot_lens) > 1):
                print "Error: more than one length for protein id: "+prot #Sanity check

            #If the length equal the canonical, this is the canonical protein id
            if (prot_lens[0] == canonic_len):
                found = True
                canonic_protein[ens_gene] = prot
                break

        if (found == False):
            print ens_gene+": Proteins don't match the Uniprot canonic"
            no_canonic_len += 1
            canonic_protein[ens_gene] = find_longest_prot(protein_ids)

ENSG00000167840.9: Proteins don't match the Uniprot canonic
ENSG00000105261.5: uniprot wasn't found for ENSG00000105261.5
ENSG00000170325.10: Proteins don't match the Uniprot canonic
ENSG00000147124.8: Proteins don't match the Uniprot canonic
ENSG00000171606.13: Proteins don't match the Uniprot canonic
ENSG00000184937.8: Proteins don't match the Uniprot canonic
ENSG00000196381.6: Proteins don't match the Uniprot canonic
ENSG00000164011.13: Proteins don't match the Uniprot canonic
ENSG00000204644.5: Proteins don't match the Uniprot canonic
ENSG00000206510.5: Proteins don't match the Uniprot canonic
ENSG00000141040.10: Proteins don't match the Uniprot canonic
ENSG00000227124.4: Proteins don't match the Uniprot canonic
ENSG00000089335.16: Proteins don't match the Uniprot canonic
ENSG00000234669.3: Proteins don't match the Uniprot canonic
ENSG00000223858.3: Proteins don't match the Uniprot canonic
ENSG00000187607.11: uniprot wasn't found for ENSG00000187607.11
ENSG00000223852.3: Proteins d

In [38]:
with open(out_path+domain+"_canonic_prot.pik", 'wb') as handle:
    pickle.dump(canonic_protein, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Printouts from creating the canonic dictionary for Zinc finger domain
ENSG00000167840.9: Proteins don't match the Uniprot canonic
ENSG00000105261.5: uniprot wasn't found for ENSG00000105261.5
ENSG00000170325.10: Proteins don't match the Uniprot canonic
ENSG00000147124.8: Proteins don't match the Uniprot canonic
ENSG00000171606.13: Proteins don't match the Uniprot canonic
ENSG00000184937.8: Proteins don't match the Uniprot canonic
ENSG00000196381.6: Proteins don't match the Uniprot canonic
ENSG00000164011.13: Proteins don't match the Uniprot canonic
ENSG00000204644.5: Proteins don't match the Uniprot canonic
ENSG00000206510.5: Proteins don't match the Uniprot canonic
ENSG00000141040.10: Proteins don't match the Uniprot canonic
ENSG00000227124.4: Proteins don't match the Uniprot canonic
ENSG00000089335.16: Proteins don't match the Uniprot canonic
ENSG00000234669.3: Proteins don't match the Uniprot canonic
ENSG00000223858.3: Proteins don't match the Uniprot canonic
ENSG00000187607.11: uniprot wasn't found for ENSG00000187607.11
ENSG00000223852.3: Proteins don't match the Uniprot canonic
ENSG00000232099.3: Proteins don't match the Uniprot canonic
ENSG00000226858.3: Proteins don't match the Uniprot canonic
ENSG00000169955.6: Proteins don't match the Uniprot canonic
ENSG00000255073.4: uniprot wasn't found for ENSG00000255073.4
ENSG00000263310.1: uniprot wasn't found for ENSG00000263310.1
ENSG00000272602.1: uniprot wasn't found for ENSG00000272602.1
ENSG00000272580.1: uniprot wasn't found for ENSG00000272580.1

In [ ]:
#Printouts from creating the canonic dictionary for Homeobox domain
ENSG00000006377.9: Proteins don't match the Uniprot canonic
ENSG00000257923.5: Proteins don't match the Uniprot canonic
ENSG00000106331.10: Proteins don't match the Uniprot canonic
ENSG00000109072.9: Proteins don't match the Uniprot canonic
ENSG00000258389.2: uniprot wasn't found for ENSG00000258389.2

In [ ]:
#Printouts from creating the canonic dictionary for WW domain
ENSG00000081026.14: Proteins don't match the Uniprot canonic
ENSG00000151276.19: Proteins don't match the Uniprot canonic
ENSG00000196504.11: Proteins don't match the Uniprot canonic

In [ ]:
#Printouts from creating the canonic dictionary for PUF domain

In [ ]:
#Printouts from creating the canonic dictionary for SH3_1 domain
ENSG00000008735.10: Proteins don't match the Uniprot canonic
ENSG00000163486.8: uniprot wasn't found for ENSG00000163486.8
ENSG00000143514.12: Proteins don't match the Uniprot canonic
ENSG00000188747.4: Proteins don't match the Uniprot canonic
ENSG00000266028.3: uniprot wasn't found for ENSG00000266028.3